In [1]:
# This produces the dataframe for WR

In [4]:
## Notes on the NFL Library ##
# the NFL python library seem to not work on Tuesday probably due to updates
# best to run this on a day other than Tues.

In [6]:
## REQUIRED ACTIONS - Include in a README doc ## 
# modify the season start date in the 'get_current_week' function
#

In [8]:
# import the libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import glob
from IPython.display import display, HTML
from datetime import datetime
import nfl_data_py as nfl
import os
import re

In [10]:
# Set Pandas options to display all columns in a single row without wrapping
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [12]:
# Function to calculate the current week of the NFL season
def get_current_week():
    current_date = datetime.now()
    season_start_date = datetime(2024, 9, 4)  # Update for the season start
    current_week = ((current_date - season_start_date).days // 7) + 1
    return current_week

# Define the current NFL year, week, and season type
current_year = datetime.now().year
current_week = get_current_week()
seasontype = 2 if current_week <= 18 else 3  # Regular season or playoffs

In [14]:
# define the years to pull
# nfl.import_weekly_data(years, columns, downcast)
def get_year_range(current_year, current_week, start_year=2017):
    if current_week <= 18:  # Regular season
        return list(range(start_year, current_year + 1))
    else:  # Playoffs
        return list(range(start_year, current_year))

# Use the function
years = get_year_range(current_year, current_week)

In [17]:
# define the base columns. 
base_columns = [
    'season', 'season_type', 'week', 'player_id', 'player_name', 
    'position', 'position_group', 'recent_team'
]

In [39]:
# Import the player IDs from nfl.import_ids() - without parameters
ids_data = nfl.import_ids()

# Drop the unnecessary columns
columns_to_drop = [
    'position', 'team', 'birthdate', 'age', 'draft_year', 
    'draft_round', 'draft_pick', 'draft_ovr', 'twitter_username', 
    'height', 'weight', 'college', 'db_season'
]
ids_data = ids_data.drop(columns=columns_to_drop, errors='ignore')

# Display the resulting dataframe for review
# print(f"Columns after dropping unnecessary ones: {ids_data.columns.tolist()}")
# display(ids_data)

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name
0,16579,49d4b627-3f19-49a8-ae33-f539e0fb1f88,23084.0,00-0039918,NaN,11560.0,NaN,4431611.0,40900.0,NaN,26695775.0,WillCa03,caleb-williams-3,17695.0,NaN,1550.0,40900.0,0.0,NaN,1286445.0,Caleb Williams,caleb williams
1,16580,d0c0630e-8925-4b81-a32b-e4146d95f01f,23046.0,00-0039851,NaN,11564.0,NaN,4431452.0,40881.0,NaN,26695757.0,MayeDr00,drake-maye-1,17673.0,NaN,1551.0,40881.0,0.0,NaN,1286803.0,Drake Maye,drake maye
2,16581,bab8c65a-9fff-410d-8e34-dc2f636d890e,22902.0,00-0039910,NaN,11566.0,NaN,4426348.0,40896.0,NaN,3123337.0,DaniJa02,jayden-daniels-1,17692.0,NaN,1552.0,40896.0,0.0,NaN,1161228.0,Jayden Daniels,jayden daniels
3,16582,f056c8ed-6ee5-492c-af11-3f7e3d97195c,22910.0,00-0039732,NaN,11563.0,NaN,4426338.0,40875.0,NaN,3121193.0,NixxBo00,bo-nix-1,16995.0,NaN,1553.0,40875.0,0.0,NaN,1161899.0,Bo Nix,bo nix
4,16583,6499a161-b221-4591-b122-d99ece1be64b,22973.0,00-0039917,NaN,11559.0,NaN,4360423.0,40889.0,NaN,2963496.0,PeniMi00,michael-penix-jr-1,17700.0,NaN,1554.0,40889.0,0.0,NaN,1107478.0,Michael Penix Jr.,michael penix
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11778,972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Doug Brien,doug brien
11779,978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jeremy Brigham,jeremy brigham
11780,984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vincent Brisby,vincent brisby
11781,990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bubby Brister,bubby brister


In [37]:
# import the weekly data from nfl.import_weekly_data(years, columns, downcast)
weekly_data = nfl.import_weekly_data(
    years=years,
    columns=base_columns
)

# display(weekly_data)

Downcasting floats.


,season,season_type,week,player_id,player_name,position,position_group,recent_team
0,2017,REG,1,00-0019596,T.Brady,QB,QB,NE
1,2017,REG,2,00-0019596,T.Brady,QB,QB,NE
2,2017,REG,3,00-0019596,T.Brady,QB,QB,NE
3,2017,REG,4,00-0019596,T.Brady,QB,QB,NE
4,2017,REG,5,00-0019596,T.Brady,QB,QB,NE
...,...,...,...,...,...,...,...,...
5376,2024,REG,10,00-0039921,T.Benson,RB,RB,ARI
5377,2024,REG,12,00-0039921,T.Benson,RB,RB,ARI
5378,2024,REG,13,00-0039921,T.Benson,RB,RB,ARI
5379,2024,REG,14,00-0039921,T.Benson,RB,RB,ARI


In [63]:
## Output: a dataframe of ALL NFL athletes info and ids since 2017

# Merge the two dataframes on 'player_id' and 'gsis_id'
# Align column names for merging
ids_data = ids_data.rename(columns={'gsis_id': 'player_id'})  
id_dataframe = pd.merge(weekly_data, ids_data, on='player_id', how='inner')

# Assign the resulting dataframe to a variable
all_players_id_data = id_dataframe

# Display the resulting ID dataframe
# display(all_players_id_data)

In [43]:
## Output: a dataframe of NFL WR info and ids since 2017
# extract WR from the dataframe
# Create a new dataframe with only wide receivers
wide_receiver_ids = all_players_id_data[all_players_id_data['position'] == 'WR']

# Display the resulting dataframe for review
# display(wide_receiver_ids)

,season,season_type,week,player_id,player_name,position,position_group,recent_team,mfl_id,sportradar_id,fantasypros_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name
104,2017,REG,1,00-0022921,L.Fitzgerald,WR,WR,ARI,7393,b6a61b38-5cfa-46eb-b1c5-b0255d7ebaf5,9383.0,1724.0,223.0,larryfitzgerald/2506106,5528.0,6762.0,1732.0,492934.0,FitzLa00,larry-fitzgerald-1,3730.0,1661.0,NaN,6762.0,246053.0,5571.0,NaN,Larry Fitzgerald,larry fitzgerald
105,2017,REG,2,00-0022921,L.Fitzgerald,WR,WR,ARI,7393,b6a61b38-5cfa-46eb-b1c5-b0255d7ebaf5,9383.0,1724.0,223.0,larryfitzgerald/2506106,5528.0,6762.0,1732.0,492934.0,FitzLa00,larry-fitzgerald-1,3730.0,1661.0,NaN,6762.0,246053.0,5571.0,NaN,Larry Fitzgerald,larry fitzgerald
106,2017,REG,3,00-0022921,L.Fitzgerald,WR,WR,ARI,7393,b6a61b38-5cfa-46eb-b1c5-b0255d7ebaf5,9383.0,1724.0,223.0,larryfitzgerald/2506106,5528.0,6762.0,1732.0,492934.0,FitzLa00,larry-fitzgerald-1,3730.0,1661.0,NaN,6762.0,246053.0,5571.0,NaN,Larry Fitzgerald,larry fitzgerald
107,2017,REG,4,00-0022921,L.Fitzgerald,WR,WR,ARI,7393,b6a61b38-5cfa-46eb-b1c5-b0255d7ebaf5,9383.0,1724.0,223.0,larryfitzgerald/2506106,5528.0,6762.0,1732.0,492934.0,FitzLa00,larry-fitzgerald-1,3730.0,1661.0,NaN,6762.0,246053.0,5571.0,NaN,Larry Fitzgerald,larry fitzgerald
108,2017,REG,5,00-0022921,L.Fitzgerald,WR,WR,ARI,7393,b6a61b38-5cfa-46eb-b1c5-b0255d7ebaf5,9383.0,1724.0,223.0,larryfitzgerald/2506106,5528.0,6762.0,1732.0,492934.0,FitzLa00,larry-fitzgerald-1,3730.0,1661.0,NaN,6762.0,246053.0,5571.0,NaN,Larry Fitzgerald,larry fitzgerald
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43449,2024,REG,2,00-0039920,M.Corley,WR,WR,NYJ,16636,bae59933-8b94-4837-990e-f0a4ced3cdbb,26023.0,NaN,11617.0,NaN,4613104.0,40944.0,NaN,3162613.0,CorlMa00,malachi-corley-1,17777.0,NaN,1607.0,40944.0,0.0,NaN,1215291.0,Malachi Corley,malachi corley
43450,2024,REG,9,00-0039920,M.Corley,WR,WR,NYJ,16636,bae59933-8b94-4837-990e-f0a4ced3cdbb,26023.0,NaN,11617.0,NaN,4613104.0,40944.0,NaN,3162613.0,CorlMa00,malachi-corley-1,17777.0,NaN,1607.0,40944.0,0.0,NaN,1215291.0,Malachi Corley,malachi corley
43451,2024,REG,10,00-0039920,M.Corley,WR,WR,NYJ,16636,bae59933-8b94-4837-990e-f0a4ced3cdbb,26023.0,NaN,11617.0,NaN,4613104.0,40944.0,NaN,3162613.0,CorlMa00,malachi-corley-1,17777.0,NaN,1607.0,40944.0,0.0,NaN,1215291.0,Malachi Corley,malachi corley
43452,2024,REG,11,00-0039920,M.Corley,WR,WR,NYJ,16636,bae59933-8b94-4837-990e-f0a4ced3cdbb,26023.0,NaN,11617.0,NaN,4613104.0,40944.0,NaN,3162613.0,CorlMa00,malachi-corley-1,17777.0,NaN,1607.0,40944.0,0.0,NaN,1215291.0,Malachi Corley,malachi corley


In [58]:
## TEST ##
# Define the season and week range for testing
test_season = 2022  # Replace with the desired season
week_start = 8      # Start of the range
week_end = 13        # End of the range

# Filter the dataframe for the specified season and week range
test_data = wide_receiver_ids[
    (wide_receiver_ids['season'] == test_season) &
    (wide_receiver_ids['week'] >= week_start) &
    (wide_receiver_ids['week'] <= week_end)
]

# Display the filtered dataframe
# display(test_data)

In [61]:
## TEST ##
# Compare player_id to gsis_id from import_ngs_data()

wr_id = '00-0038976'  # Replace with the desired player_id

# Filter the dataframe for the matching player_id
player_data = wide_receiver_ids[wide_receiver_ids['player_id'] == wr_id]

# Display the player's weekly data
# display(player_data)

In [70]:
# WR-specific columns (receiving-related)
wr_columns = [
    'receptions', 'targets', 'receiving_yards', 'receiving_tds',
    'receiving_fumbles', 'receiving_fumbles_lost',
    'receiving_air_yards', 'receiving_yards_after_catch',
    'receiving_first_downs', 'receiving_epa',
    'receiving_2pt_conversions', 'racr', 'target_share',
    'air_yards_share', 'wopr'
]

# Pull WR-specific columns from weekly data
wr_stats = nfl.import_weekly_data(
    years=years,
    columns=['player_id', 'season', 'week'] + wr_columns  # Include keys for merging
)

# Merge WR-specific stats with wide_receiver_ids
wr_ids_weekly_stats = pd.merge(
    wide_receiver_ids,
    wr_stats,
    on=['player_id', 'season', 'week'],  # Ensure correct alignment
    how='inner'
)

# Display the resulting dataframe for review
print(f"Shape of merged dataframe: {wr_ids_weekly_stats.shape}")
# display(wr_ids_weekly_stats.head())
display(wr_ids_weekly_stats)


Downcasting floats.
Shape of merged dataframe: (17328, 44)


,season,season_type,week,player_id,player_name,position,position_group,recent_team,mfl_id,sportradar_id,fantasypros_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr
0,2017,REG,1,00-0022921,L.Fitzgerald,WR,WR,ARI,7393,b6a61b38-5cfa-46eb-b1c5-b0255d7ebaf5,9383.0,1724.0,223.0,larryfitzgerald/2506106,5528.0,6762.0,1732.0,492934.0,FitzLa00,larry-fitzgerald-1,3730.0,1661.0,NaN,6762.0,246053.0,5571.0,NaN,Larry Fitzgerald,larry fitzgerald,6,13,74.0,0,0.0,0.0,144.0,44.0,4.0,0.997088,0,0.513889,0.276596,0.342043,0.654324
1,2017,REG,2,00-0022921,L.Fitzgerald,WR,WR,ARI,7393,b6a61b38-5cfa-46eb-b1c5-b0255d7ebaf5,9383.0,1724.0,223.0,larryfitzgerald/2506106,5528.0,6762.0,1732.0,492934.0,FitzLa00,larry-fitzgerald-1,3730.0,1661.0,NaN,6762.0,246053.0,5571.0,NaN,Larry Fitzgerald,larry fitzgerald,3,6,21.0,0,0.0,0.0,29.0,17.0,2.0,-3.455533,0,0.724138,0.166667,0.069378,0.298565
2,2017,REG,3,00-0022921,L.Fitzgerald,WR,WR,ARI,7393,b6a61b38-5cfa-46eb-b1c5-b0255d7ebaf5,9383.0,1724.0,223.0,larryfitzgerald/2506106,5528.0,6762.0,1732.0,492934.0,FitzLa00,larry-fitzgerald-1,3730.0,1661.0,NaN,6762.0,246053.0,5571.0,NaN,Larry Fitzgerald,larry fitzgerald,13,15,149.0,1,0.0,0.0,138.0,45.0,6.0,7.632769,0,1.079710,0.312500,0.369973,0.727731
3,2017,REG,4,00-0022921,L.Fitzgerald,WR,WR,ARI,7393,b6a61b38-5cfa-46eb-b1c5-b0255d7ebaf5,9383.0,1724.0,223.0,larryfitzgerald/2506106,5528.0,6762.0,1732.0,492934.0,FitzLa00,larry-fitzgerald-1,3730.0,1661.0,NaN,6762.0,246053.0,5571.0,NaN,Larry Fitzgerald,larry fitzgerald,4,7,32.0,1,0.0,0.0,31.0,18.0,1.0,0.162141,0,1.032258,0.137255,0.070938,0.255539
4,2017,REG,5,00-0022921,L.Fitzgerald,WR,WR,ARI,7393,b6a61b38-5cfa-46eb-b1c5-b0255d7ebaf5,9383.0,1724.0,223.0,larryfitzgerald/2506106,5528.0,6762.0,1732.0,492934.0,FitzLa00,larry-fitzgerald-1,3730.0,1661.0,NaN,6762.0,246053.0,5571.0,NaN,Larry Fitzgerald,larry fitzgerald,6,10,51.0,0,0.0,0.0,44.0,29.0,5.0,2.428232,0,1.159091,0.227273,0.105516,0.414770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17323,2024,REG,2,00-0039920,M.Corley,WR,WR,NYJ,16636,bae59933-8b94-4837-990e-f0a4ced3cdbb,26023.0,NaN,11617.0,NaN,4613104.0,40944.0,NaN,3162613.0,CorlMa00,malachi-corley-1,17777.0,NaN,1607.0,40944.0,0.0,NaN,1215291.0,Malachi Corley,malachi corley,1,1,4.0,0,0.0,0.0,-1.0,5.0,0.0,-0.475780,0,0.000000,0.034483,-0.006579,0.047119
17324,2024,REG,9,00-0039920,M.Corley,WR,WR,NYJ,16636,bae59933-8b94-4837-990e-f0a4ced3cdbb,26023.0,NaN,11617.0,NaN,4613104.0,40944.0,NaN,3162613.0,CorlMa00,malachi-corley-1,17777.0,NaN,1607.0,40944.0,0.0,NaN,1215291.0,Malachi Corley,malachi corley,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN
17325,2024,REG,10,00-0039920,M.Corley,WR,WR,NYJ,16636,bae59933-8b94-4837-990e-f0a4ced3cdbb,26023.0,NaN,11617.0,NaN,4613104.0,40944.0,NaN,3162613.0,CorlMa00,malachi-corley-1,17777.0,NaN,1607.0,40944.0,0.0,NaN,1215291.0,Malachi Corley,malachi corley,1,2,2.0,0,0.0,0.0,12.0,0.0,1.0,-2.246118,0,0.166667,0.060606,0.057692,0.131294
17326,2024,REG,11,00-0039920,M.Corley,WR,WR,NYJ,16636,bae59933-8b94-4837-990e-f0a4ced3cdbb,26023.0,NaN,11617.0,NaN,4613104.0,40944.0,NaN,3162613.0,CorlMa00,malachi-corley-1,17777.0,NaN,1607.0,40944.0,0.0,NaN,1215291.0,Malachi Corley,malachi corley,1,1,10.0,0,0.0,0.0,10.0,0.0,0.0,0.563583,0,1.000000,0.034483,0.080645,0.108176


In [53]:
# Output: imports the NFL next-generation stats from the nfl python library

# import the next generation stats (NGS) from nfl.import_ngs_data()
# note: ngs starts at week 0 (previous season totals) - not needed so drop those rows

## Pull NGS receiving data for the specified years
# ngs_wr_df = nfl.import_ngs_data('receiving', years)

# Exclude rows where 'week' == 0 and filter for 'WR' position in one step
# ngs_wr_df = ngs_wr_df[(ngs_wr_df['week'] != 0) & (ngs_wr_df['player_position'] == 'WR')]

# Display the resulting dataframe
# display(ngs_wr_df)

,season,season_type,week,player_display_name,player_position,team_abbr,avg_cushion,avg_separation,avg_intended_air_yards,percent_share_of_intended_air_yards,receptions,targets,catch_percentage,yards,rec_touchdowns,avg_yac,avg_expected_yac,avg_yac_above_expectation,player_gsis_id,player_first_name,player_last_name,player_jersey_number,player_short_name
1725,2017,REG,1,Ryan Grant,WR,WAS,9.936667,2.894592,4.410000,7.154639,4,6,66.666667,61.0,0,11.232500,10.072361,1.160139,00-0031068,Ryan,Grant,14,R.Grant
1726,2017,REG,1,Martavis Bryant,WR,PIT,8.300000,4.122054,12.688333,33.327496,2,6,33.333333,14.0,0,0.155000,4.098278,-3.943278,00-0031373,Martavis,Bryant,10,M.Bryant
1729,2017,REG,1,Jamison Crowder,WR,WAS,7.655000,3.177793,10.540000,19.949707,3,7,42.857143,14.0,0,1.450000,1.631897,-0.181897,00-0031941,Jamison,Crowder,80,J.Crowder
1732,2017,REG,1,Nelson Agholor,WR,PHI,7.423750,2.462620,10.463750,20.274656,6,8,75.000000,86.0,1,5.611667,3.262470,2.349197,00-0031549,Nelson,Agholor,13,N.Agholor
1733,2017,REG,1,John Brown,WR,ARI,7.360000,2.751526,13.422222,28.208481,4,9,44.444444,32.0,0,-0.377500,0.961993,-1.339493,00-0031051,John,Brown,12,J.Brown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13269,2024,REG,18,Marquez Valdes-Scantling,WR,NO,3.935714,2.010495,15.492500,38.403619,3,8,37.500000,33.0,0,6.613333,7.086589,-0.473256,00-0034272,Marquez,Valdes-Scantling,10,M.Valdes-Scantling
13270,2024,REG,18,Xavier Legette,WR,CAR,3.856000,2.837124,8.482000,14.529446,3,5,60.000000,30.0,0,4.970000,2.999472,1.970528,00-0039342,Xavier,Legette,17,X.Legette
13271,2024,REG,18,DK Metcalf,WR,SEA,3.836000,1.329995,19.976000,45.638565,3,5,60.000000,53.0,1,3.253333,1.865671,1.387663,00-0035640,DeKaylin,Metcalf,14,D.Metcalf
13272,2024,REG,18,Michael Pittman,WR,IND,3.758000,2.731566,12.983000,35.302915,6,10,60.000000,72.0,0,1.731667,1.706524,0.025143,00-0036252,Michael,Pittman,11,M.Pittman


In [ ]:
# Ouput:
# Joins wr_ids_weekly_stats dataframe with ngs_wr_df using the keys 'player_id' and 'player_gsis_id'
